# BUILD LLM Bootcamp Day 2

### Install Libraries

In [ ]:
!pip install --no-deps --force-reinstall snowflake_ml_python-1.0.12-py3-none-any.whl

In [ ]:
!pip install peft transformers==4.34.0 tokenizers vllm==0.2.1.post1 bitsandbytes datasets absl-py==1.3.0

In [ ]:
# TODO: Set your Hugging Face token
!huggingface-cli login --token 

### Import Libraries

In [ ]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
import snowflake.ml.playground.finetune as ft_util
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Establish Secure Connection

*NOTE: Update [connection.json](connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [ ]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Fine-tune Llama 2 in Snowpark Container Services

#### Load Training Dataset

*NOTE: Fine-tuning using 100 records on this setup (compute and other resources) will take about ~5mins.*

In [ ]:
df = pd.read_json("transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

In [ ]:
train_df.head()

#### Fine-tune Llama 2 Model

Fine-tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve for a concrete task based on task specific data. Today we are going to fine-tune 7B Llama 2 model using LoRA (Low-Rank Adaptation)--which is a parameter efficient way of fine-tuning LLM. 

Instead of adjusting all the ~7B parameters, LoRA allows us to adjust only a percent of model weights--which can save compute and memory resources dramatically. For this lab, we will fine-tune our model using LoRA on a single A10 GPU. This will demostrate how good the inference can be on fine-tuned models even with limited compute.

##### Configuration

We're passing `train_dataset` and `eval_dataset` that are used to generate loss calculation during fine-tuning process and we've set `output_weights_dir` as the directory where the fine-tuned weights will be stored after fine-tuning job completes.

We're also setting `max_steps=1` with 4bit quantization. Quantization is required to train on single A10 GPU. To achieve good performance for the task, you will need at least 1 `num_epochs`, feel free to explore this on your own.

In [ ]:
!mkdir merged_weights_dir
!mkdir output_weights_dir

In [ ]:
%%time
# NOTE: Fine-tuning using 100 records on this setup (compute and other resources) will take about ~5mins.

merged_weights_dir = 'merged_weights_dir'
output_weights_dir = 'output_weights_dir'

runner = ft_util.FinetuneModelRunner(
    base_model_id="meta-llama/Llama-2-7b-chat-hf",
    task_type=ft_util.FinetuneTaskType.INSTRUCT,
    train_dataset=train_df,
    eval_dataset=eval_df,
)

mcf = ft_util.ModelConfig(
    max_steps=1,
    quantization=ft_util.QuantizationConfig(
        k_bits=4,
    ),
    merged_weights_dir_path=merged_weights_dir,
)

_ = runner.train(output_weights_dir, model_cfg=mcf)

#### Evaluate Results on Eval Dataset

After the fine-tuning job completes, we call `get_eval_results`, this will run evaluation and get model predictions using the entire `eval_dataset`. You can see the output by model in the `predicted` column of the dataframe.

*Examine the "output" and "predicted" columns*

In [ ]:
eval_df = runner.get_eval_results()
eval_df.head()

#### Inference on Custom Transcript

Additionally, you may want to manually craft some cases to test the boundaries of the model. To do that, you can use the `infer` method to run prediction against a single sample interactively.

In [ ]:
runner.infer(
     {
    'instruction': """
        "Extract JSON response with 'location' and 'toy_list' as keys. 'location': 
        Location of the caller. Include city only.'toy_list': List of toy names from the caller."
    """,
    'input': """
        "frosty: Hello, hello! Who's ready to chat with Frosty today?\ncaller: Hi, I'm Dustin from Cape Town, South Africa.\nfrosty: 
        Hi Dustin, it's awesome to meet you! Do you already know what you would like for your holiday gift?\ncaller: 
        Yeah, I think I want the ninja turtles delivery van, or maybe the robot dog. 
        I can't decide...\nfrosty: Both sound like entertaining choices! 
        Why are you interested in these particular toys?\ncaller: The ninja turtles van is so cool! And the robot dog, it's like a pet you don't have to clean up after.\nfrosty: 
        That's true! Savage choices, Dustin! How do you usually celebrate the holiday season?\ncaller: 
        We usually have a big family barbeque and play games all day long.\nfrosty: Sounds fun! What is your favorite thing about this time of year?
        \ncaller: I like that everyone is happy and I get to spend a lot of time with my family.
        \nfrosty: That's wonderful! So Dustin, have you thought any more about your holiday wish?
        \ncaller: I think I'm going to get the robot dog. It sounds fun and I won't have to walk it.
        \nfrosty: That's a clever choice, Dustin! I hope you have a fantastic holiday season!"
"""
    }
)

#### Batch Inference on Test Dataset

For batch inference to test on a holdout dataset, you can use `infer_batch` on a dataframe.

*Examine the "output" and "predicted" columns*

In [ ]:
test_res = runner.infer_batch(test_df)
test_res.head()

### Log and Deploy Fine-tuned Llama 2

In [ ]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "FineTunedV1.0"
DEPLOYMENT_NAME = "FINETUNED_LLAMA2"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

#### Delete any existing deployments and models from in Snowpark registry to avoid resource contention during this bootcamp

In [ ]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

In [ ]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [ ]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

### Log and Deploy Fine-tuned Llama 2

- Logging and deploying fine-tuned model on this setup (compute and other resources) will take about ~15mins
- Notice that the "model_id_or_path" is set to the output weights folder `output_weights_dir` where the weights from fine-tuning the model are stored

In [ ]:
%%time

options = llm.LLMOptions(
    token=connection_parameters["huggingface_token"],
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path=output_weights_dir,
    options=options
)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

#### Prepare Eval Dataset with Specific Instructions

In [ ]:
begin_prompt = \
"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys.
'location': Location string of the caller.
'toy_list': List of toy names from the caller.
### Input:

"""
end_prompt = " [/INST]"

eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt
eval_df = eval_df.reset_index(drop=True)
eval_df.head(1)

#### Inference on Eval Dataset using fine-tuned Llama 2

In [ ]:
llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df).head()

#### Run Inference in SQL

In [ ]:
# TODO: Go to Snowsight and run the following commands in a SQL Worksheet

# create or replace function build_prompt(inp varchar)
# returns varchar
# language sql
# as $$
#     '[INST] <<SYS>>\n' ||
#     'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n' ||
#     '<</SYS>>\n' ||
#     '### Instruction:\n' ||
#     'Extract JSON response with \'location\' and \'toy_list\' as keys.\n' ||
#     '\'location\': Location of the caller. Include city only.\n' ||
#     '\'toy_list\': List of toy names from the caller.\n' ||
#     '### Input:\n' ||
#     '"' || inp || '"\n' ||
#     '[/INST]'
# $$;

# SELECT TRAINED_LLAMA2(object_construct('input', build_prompt(select transcript from frosty_transcripts limit 1)));

#### Clean Up Resources

Delete deployment and the model

In [ ]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)